In [35]:
# !sudo docker run -p 8080:8080 -v /tmp:/tmp --device /dev/dri intel/video-analytics-serving

Run the server first:
```
sudo docker run -u root -p 8080:8080 -it --device /dev/dri --privileged -v /opt/intel/openvino_2021.4/deployment_tools/open_model_zoo/tools/downloader/:/home/dlstreamer/intel/dl_streamer/models -v /tmp/:/tmp/ -e DISPLAY=:0 --rm intel/dlstreamer-pipeline-server:2022.2.0-ubuntu20
```

In [36]:
!git clone https://github.com/intel-iot-devkit/sample-videos.git  /tmp/sample-videos

fatal: destination path '/tmp/sample-videos' already exists and is not an empty directory.


In [37]:
#concurrency test
import os, time
import requests
import json,ast
import random,string
req_di={}
def check_res(url=None,info=False):
    if url is None:
        return
    payload = {}
    headers = {
        'Content-Type': 'application/json'
    }
    # print(payload)
    # response = requests.post(url, headers=headers, data=json.dumps(payload))
    response = requests.get(url, headers=headers, json=payload)
    if info:
        print("Status Code:", response.status_code)
        print("Response Body:", response.text)
    res=ast.literal_eval(response.text)
    return res["elapsed_time"] if res["state"]=="COMPLETED" else None

def req_va(video_path, result_path=None, fun_url="http://localhost:8080/pipelines/object_detection/person_vehicle_bike"):
    url = fun_url

    payload = {
        "source": {
            "uri": "file:///tmp/traffic.mp4",
            "type": "uri"
        },
        "destination": {
            "metadata": {
                "type": "file",
                "path": "/tmp/results2.txt",
                "format": "json-lines"
            }
        }
    }
    payload["source"]["uri"]="file://"+video_path
    if result_path is None:
        # result_path="/tmp/"+video_path.split("/")[-1].replace(".mp4",".log")
        hash_string = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
        result_path="/tmp/"+video_path.split("/")[-1].replace(".mp4","")
        result_path=result_path+"_"+hash_string+".log"
    payload["destination"]["metadata"]["path"]=result_path
    headers = {
        'Content-Type': 'application/json'
    }
    print(payload)
    # response = requests.post(url, headers=headers, data=json.dumps(payload))
    response = requests.post(url, headers=headers, json=payload)

    print("Status Code:", response.status_code)
    print("Response Body:", response.text)
    return response.text

# videos=['one-by-one-person-detection.mp4', 'store-aisle-detection.mp4', 'people-detection.mp4', 'bottle-detection.mp4', 'driver-action-recognition.mp4', 'head-pose-face-detection-male.mp4', 'head-pose-face-detection-female.mp4', 'car-detection.mp4', 'face-demographics-walking-and-pause.mp4', 'worker-zone-detection.mp4', 'bolt-multi-size-detection.mp4', 'bolt-detection.mp4', 'head-pose-face-detection-female-and-male.mp4', 'classroom.mp4', 'person-bicycle-car-detection.mp4', 'face-demographics-walking.mp4', 'fruit-and-vegetable-detection.mp4']
int_key=['0-F3q8Aj9Zk', '01_BrVxYsE0', '0-CYG7bH54A', '0B4T3GIkMZw', '0CZuZZ17mck', '08uNrWxxHGY', '0KqeKi2CBqg', '06ofnvq2Hjs', '0O589B9zr84', 'classroom', '0Qed9-9yp9Q', 'bottle-detection', '0DsdxtAvHDk', '04LdesS7Pxk', '0EepbsAtiDk', '0IFJNGqjsUQ', '0FTaJwbFdZI']
final_ans={}
cnt1=0
ha_ids=[]
# invoke
for con in range(1):
    for vi in ["classroom"]:
    # for vi in int_key:
        if vi == "bottle-detection" or vi == "classroom":
            vi_p=os.path.join("/tmp/sample-videos/",f"{vi}.mp4")
        else:
            vi_p=os.path.join("/tmp/ActivityNet/",f"{vi}.mp4")
        if not os.path.exists(vi_p):
            continue
        # print(vi_p)
        ha_id=req_va(vi_p)
        ha_id=ha_id.strip().replace('"','')
        ha_ids.append(ha_id)
        # req_di[ha_id]=vi

        # while(True):
        #     time.sleep(5)
        #     name="object_detection"
        #     version="person_vehicle_bike"
        #         # instance_id="f8dbf95e87d211efaf7c0242ac110002"
        #     url=f"http://localhost:8080/pipelines/{name}/{version}/{ha_id}/status"
        #         # GET /pipelines/{name}/{version}/{instance_id}
        #     e2e_tim=check_res(url)
        #     if e2e_tim is not None:
        #         final_ans[vi]=e2e_tim
        #         break
        # if cnt1 >=3:
        #     break
        # cnt1+=1
    
# print(req_di)
# print(final_ans)


{'source': {'uri': 'file:///tmp/sample-videos/classroom.mp4', 'type': 'uri'}, 'destination': {'metadata': {'type': 'file', 'path': '/tmp/classroom_zRf4uomN.log', 'format': 'json-lines'}}}
Status Code: 200
Response Body: "3a7587ce8dc611efb8fb0242ac110002"



In [38]:
len(ha_ids)

1

In [39]:

res0={}
cout=0
while(True):
    time.sleep(5)
    name="object_detection"
    version="person_vehicle_bike"
    flag=1
    for id0 in ha_ids:
        url=f"http://localhost:8080/pipelines/{name}/{version}/{id0}/status"
            # GET /pipelines/{name}/{version}/{instance_id}
        e2e_tim=check_res(url)
        if e2e_tim is not None:
            # cout+=1
            # print(cout,e2e_tim)
            res0[id0]=e2e_tim
            # break
        else:
            flag=0
            time.sleep(5)
    if flag==1:
        break
    # break


In [40]:
print(res0.values())
# for k,v in res0.items():
#     if v < 1:
#         print(k)

dict_values([8.67960262298584])


In [41]:
dlstreamer_10g=[56.1298394203186, 59.87207388877869, 62.47958445549011, 64.35585594177246, 63.425336837768555, 61.63944053649902, 59.30074977874756, 56.61838889122009, 64.00640034675598, 64.16362142562866]
dlstreamer_20g=[55.86392283439636, 59.98772478103638, 62.54295301437378, 64.30032801628113, 64.48698616027832, 63.35225033760071, 61.72321677207947, 59.31234407424927, 64.60616064071655, 56.36464548110962]

dlstreamer_5g=[54.22969436645508, 60.5618691444397, 58.04087805747986, 62.640612840652466, 63.37082290649414, 63.069164752960205, 62.3219735622406, 58.999905824661255, 53.91446542739868, 48.695799827575684]
dlstreamer_5g_2=[59.5600323677063, 55.708837032318115, 62.46635413169861, 63.734591007232666, 64.43418025970459, 64.30828595161438, 63.586267709732056, 55.384522914886475, 50.25426244735718, 60.146037340164185]

dlstreamer_4g=[48.91807723045349, 55.905741453170776, 66.96529579162598, 58.6130154132843, 60.62659287452698, 64.59975075721741, 60.08772563934326, 49.35785102844238, 62.87034201622009, 56.58643865585327]
dlstreamer_3_5g=[491.1393151283264, 495.68970370292664, 497.9567837715149, 498.33028769493103, 497.72154426574707, 496.80108094215393, 497.17349076271057, 495.45061683654785, 492.51891040802, 490.43295097351074]

# 3g OOM

g8i_10g=[81.73466849327087, 80.46101236343384, 78.59616279602051, 71.08804631233215, 76.70246911048889, 79.98972177505493, 81.74080967903137, 82.50462079048157, 82.59852933883667, 75.76524782180786]
g8i_40g=[70.27128529548645, 81.46059203147888, 81.33707571029663, 78.10851073265076, 75.33292984962463, 76.45837473869324, 79.41021847724915, 82.31308364868164, 82.20492434501648, 79.98784732818604]
g8i_8g=[81.98658442497253, 79.84051990509033, 75.1294813156128, 76.1957643032074, 70.47364950180054, 79.59478688240051, 81.46420669555664, 82.04936051368713, 81.16118693351746, 77.7790298461914]
g8i_5g_5=[41.42081689834595, 41.15780568122864, 39.02669072151184, 41.0284526348114, 40.504769802093506]
g8i_5g_10=[71.16168642044067, 82.55540823936462, 81.93622303009033, 78.35867643356323, 75.65382075309753, 77.00379633903503, 80.1970841884613, 82.00364947319031, 82.53348779678345, 80.40122318267822]
g8i_5g_3=[24.01544451713562, 24.362300872802734, 24.523056507110596]
g8i_5g_1=[8.67960262298584]


In [42]:
# id0="83c8013e8a0811efab3d0242ac110002"
# url=f"http://localhost:8080/pipelines/{name}/{version}/{id0}/status"
#             # GET /pipelines/{name}/{version}/{instance_id}
# e2e_tim=check_res(url,True)
